# Banco Lyz — Analise de Concessao de Credito

In [7]:
#def -> cria função

def input_validado(msg, tipo=int, minimo=None, maximo=None):
    """Le e valida a entrada do usuario.
    Repete o pedido ate receber um valor do tipo correto e dentro do intervalo.
    """
    while True:
        try:
            v = tipo(input(msg))
            if (minimo is not None and v < minimo) or (maximo is not None and v > maximo):
                print(f"Valor deve estar entre {minimo} e {maximo}.")
            else:
                return v
        except ValueError:
            print("Entrada invalida! Digite apenas numeros.")


def classificar_score(score):
    """Retorna o perfil de risco do cliente com base no score de credito.
    Faixas: >700 excelente | >500 bom pagador | >300 risco medio | resto risco alto.
    """
    if score > 700: return "excelente"
    elif score > 500: return "bom pagador"
    elif score > 300: return "risco medio"
    else: return "risco alto"


def calcular_taxa(pontuacao):
    """Calcula a taxa de juros mensal dinamicamente com base na pontuacao composta.
    Quanto menor a pontuacao, maior a taxa: varia de 1.5% a.m. (melhor) ate 4.5% a.m. (pior).
    Usa interpolacao linear entre os extremos.
    """
    TAXA_MIN, TAXA_MAX = 1.5, 4.5  # percentual a.m.
    # pontuacao 100 -> taxa 1.5% | pontuacao 0 -> taxa 4.5%
    return TAXA_MAX - (pontuacao / 100) * (TAXA_MAX - TAXA_MIN)


def calcular_parcela(valor, taxa_mensal, meses):
    """Calcula o valor da parcela pelo sistema Price (juros compostos).
    Retorna None se meses for zero.
    """
    if meses == 0:
        return None
    t = taxa_mensal / 100
    return valor * (t * (1 + t) ** meses) / ((1 + t) ** meses - 1)


def verificar_comprometimento(parcela, renda, limite_pct=0.30):
    """Verifica se a parcela compromete mais de limite_pct (padrao 30%) da renda mensal.
    Retorna True se estiver dentro do limite aceitavel.
    """
    return parcela <= renda * limite_pct

In [8]:
# --- Coleta de dados ---
print("ANALISE DE CONCESSAO DE CREDITO - Banco Lyz")

score            = input_validado("Score de credito (0-1000): ",        int,   0,    1000)
tempo_conta      = input_validado("Tempo de conta corrente (meses): ",  int,   0,    600)
tempo_trabalho   = input_validado("Tempo de trabalho (meses): ",        int,   0,    600)
renda            = input_validado("Renda mensal (R$): ",                float, 0.0,  None)
valor_solicitado = input_validado("Valor do credito solicitado (R$): ", float, 0.01, None)
prazo_meses      = input_validado("Prazo desejado (meses): ",           int,   1,    360)

ANALISE DE CONCESSAO DE CREDITO - Banco Lyz


In [9]:
# --- Calculo da pontuacao composta (0-100) ---
# Cada criterio e normalizado para 0-100 e ponderado pelo seu peso
nota_score    = (score / 1000) * 100
nota_conta    = min(tempo_conta / 60, 1) * 100     # referencia: 5 anos (60 meses)
nota_trabalho = min(tempo_trabalho / 60, 1) * 100  # referencia: 5 anos
nota_renda    = min(renda / 10000, 1) * 100        # referencia: R$ 10.000

pontuacao = nota_score * 0.4 + nota_conta * 0.2 + nota_trabalho * 0.2 + nota_renda * 0.2

# --- Limite, taxa e parcela ---
limite         = renda * 6 * (pontuacao / 100)   # limite: ate 6x a renda, ajustado pela pontuacao
taxa           = calcular_taxa(pontuacao)          # taxa dinamica baseada na pontuacao
valor_aprovado = min(valor_solicitado, limite)     # valor efetivo a ser financiado
parcela        = calcular_parcela(valor_aprovado, taxa, prazo_meses)
perfil         = classificar_score(score)

# --- Verificacao de comprometimento de renda (limite: 30%) ---
renda_ok = verificar_comprometimento(parcela, renda)

# --- Resultado ---
# f no print concatena dados
print(f"\nPerfil       : {perfil.upper()}")
print(f"Pontuacao    : {pontuacao:.1f}/100")
print(f"Taxa de juros: {taxa:.2f}% a.m.")
print(f"Limite       : R$ {limite:,.2f}")
print(f"Solicitado   : R$ {valor_solicitado:,.2f}")
print(f"Parcela est. : R$ {parcela:,.2f} / mes ({prazo_meses}x)")
print(f"Comprometim. : {(parcela/renda*100):.1f}% da renda {'[OK]' if renda_ok else '[ACIMA DO LIMITE]'}")
print("-" * 45)

if pontuacao < 40:
    print("CREDITO NEGADO — Pontuacao insuficiente.")
elif not renda_ok:
    print("CREDITO NEGADO — Parcela compromete mais de 30% da renda.")
elif valor_solicitado > limite:
    print(f"CREDITO PARCIAL — Valor aprovado: R$ {limite:,.2f}")
else:
    print(f"CREDITO APROVADO — R$ {valor_solicitado:,.2f}")

print(f"\nNotas: score={nota_score:.1f}(40%) | conta={nota_conta:.1f}(20%) | trabalho={nota_trabalho:.1f}(20%) | renda={nota_renda:.1f}(20%)")


Perfil       : BOM PAGADOR
Pontuacao    : 31.6/100
Taxa de juros: 3.55% a.m.
Limite       : R$ 4,546.56
Solicitado   : R$ 300.00
Parcela est. : R$ 18.79 / mes (24x)
Comprometim. : 0.8% da renda [OK]
---------------------------------------------
CREDITO NEGADO — Pontuacao insuficiente.

Notas: score=58.6(40%) | conta=13.3(20%) | trabalho=3.3(20%) | renda=24.0(20%)
